In [1]:
#
import dolfin as df
import numpy as np
import matplotlib.pyplot as plt
import time

from pfbase_ali_mod import *
#from pfbase import *
from ufl import split, dx, inner, grad, variable, diff

save_solution = True

###################################
# Optimization options for the finite element form compiler
###################################
df.parameters["form_compiler"]["cpp_optimize"] = True
df.parameters['form_compiler']['cpp_optimize_flags'] = '-O3 -ffast-math'
df.parameters["form_compiler"]["quadrature_degree"] = 3

In [9]:
###################################
# Create or read mesh
###################################
Lx = Ly = 100.0
Nx = Ny = 50
mesh = df.RectangleMesh(df.Point(0.0, 0.0), df.Point(Lx, Ly), Nx, Ny, 'crossed')

In [10]:

###################################
# Model Setup - need
#   dt, w, w0, F, J, bcs
###################################
dt = df.Constant(1e-1)

# CH parameters
c_alpha = df.Constant(0.3)
c_beta = df.Constant(0.7)
kappa = df.Constant(2.0)
rho = df.Constant(5.0)
M = df.Constant(5.0)

# also Poisson equation parameters
k = df.Constant(0.09)    
epsilon = df.Constant(90.0)

# FEM setup
P1 = df.FunctionSpace(mesh, 'P', 1)
PE = P1.ufl_element()
ME = [PE, PE, PE, PE, PE]
ME = df.MixedElement(ME)
W  = df.FunctionSpace(mesh,  ME)

w  = df.Function(W)
dw = df.TrialFunction(W)
w_ = df.TestFunction(W)

# Initial conditions
cc0 = 0.5
cc1 = 0.04
ux0 = 0.0
uy0 = 0.0

w0 = df.Function(W)
w_ic = InitialConditionsBench_ali(cc0, cc1, degree=2)
w0.interpolate(w_ic)

# Free energy functional
c, _, phi, ux, uy = df.split(w)
c   = df.variable(c)
phi = df.variable(phi)




In [11]:
# this function returns the total strain tensor components
# vx = x component of the displacement vector, vy = y component of the displacement
def eps(vx, vy):
    duxdx, duxdy = df.grad(vx)
    duydx, duydy = df.grad(vy)
    eps_xx = duxdx
    eps_xy = 0.5*(duxdy+duydx) # eps_yx = eps_xy
    eps_yy = duydy
    return eps_xx, eps_xy, eps_yy 

#plane stress case
def sigma(vx, vy, c):
    h_temp = h(c)
    modulus_mod = (1+0.1*h_temp) 
    c_1111 = 250 * modulus_mod
    c_1122 = 150 * modulus_mod
    c_1212 = 100 * modulus_mod
    eps_chem = 0.005
    eps_xx, eps_xy, eps_yy = eps(vx, vy)
    
    #obtain elastic strain term
    eps_el_xx = eps_xx-eps_chem*h_temp
    eps_el_yy = eps_yy-eps_chem*h_temp
    eps_el_xy = eps_xy
    
    sigma_xx = c_1111*eps_el_xx+c_1122*eps_el_yy #subtract out chemical contribution
    sigma_xy = c_1212*eps_el_xy*2                # sigma_yx = sigma_xy
    sigma_yy = c_1111*eps_el_yy+c_1122*eps_el_xx
    
    return sigma_xx, sigma_xy, sigma_yy


#elastic free energy contribution
def f_el(vx, vy, c):
    sigma_xx, sigma_xy, sigma_yy = sigma(vx, vy, c)
    eps_xx, eps_xy, eps_yy = eps(vx, vy)
    return 0.5*(eps_xx*sigma_xx+eps_xy*sigma_xy*2+eps_yy*sigma_yy)


def F_ux_weak_form(ux_, sigma_xx, sigma_xy):
    
    sigma_xx_x, sigma_xx_y = df.grad(sigma_xx)
    sigma_xy_x, sigma_xy_y = df.grad(sigma_xy)

    lhs =  ux_*(sigma_xx_x+sigma_xy_x)*dx
    rhs = 0

    F = lhs - rhs

    return F

def F_uy_weak_form(uy_, sigma_yy, sigma_xy):
    
    sigma_yy_x, sigma_yy_y = df.grad(sigma_yy)
    sigma_xy_x, sigma_xy_y = df.grad(sigma_xy)

    lhs =  uy_*(sigma_yy_y+sigma_xy_y)*dx
    rhs = 0

    F = lhs - rhs

    return F

In [12]:
sigma_xx, sigma_xy, sigma_yy = sigma(w[3], w[4], c)

# chemical free energy density for benchmark 4
a_0 = df.Constant(0)
a_1 = df.Constant(0)
a_2 = df.Constant(8.072789087)
a_3 = df.Constant(-81.24549382)
a_4 = df.Constant(408.0297321)
a_5 = df.Constant(-1244.129167)
a_6 = df.Constant(2444.046270)
a_7 = df.Constant(-3120.635139)
a_8 = df.Constant(2506.663551)
a_9 = df.Constant(-1151.003178)
a_10 = df.Constant(230.2006355)

#barrier height for benchmark 4
wall = 0.1

# CH term for benchmark 4
kappa = 0.29

f_chem = wall*(a_0+a_1*c+a_2*(c**2)+a_3*(c**3)+a_4*(c**4)
            +a_5*(c**5)+a_6*(c**6)+a_7*(c**7)+a_8*(c**8)
            +a_9*(c**9)+a_10*(c**10))


In [26]:
f_chem = rho * (c - c_alpha)**2 * (c_beta - c)**2
f_elec = k * c * phi / 2.0
f_elec = df.Constant(0.0)
f_m = f_el(ux, uy, c)

dfdc = df.diff(f_chem, c) + k * phi + df.diff(f_m, c)

## weak form
Fc = cahn_hilliard_weak_form(w[0], w[1], w_[0], w_[1], w0[0], dt, M, kappa, dfdc)
Fp = poisson_weak_form(w[2], w_[2], -k * c / epsilon, df.Constant(1.0))
F_mx = F_ux_weak_form(w_[3], sigma_xx, sigma_xy)
F_my = F_uy_weak_form(w_[4], sigma_yy, sigma_xy)

F = Fc + Fp + F_mx + F_my

# BC
tol = 1E-12
def boundary_left(x, on_boundary):
    return on_boundary and df.near(x[0], 0, tol)

def boundary_right(x, on_boundary):
    return on_boundary and df.near(x[0], Lx, tol)


In [27]:
phi_right = df.Expression(("sin(x[1]/7)"), degree=2)

_, _, Wphi, Wux, Wuy = W.split()
bc_phi_left  = df.DirichletBC(Wphi, df.Constant(0.0), boundary_left)
bc_phi_right = df.DirichletBC(Wphi, phi_right, boundary_right)

bc_ux_left  = df.DirichletBC(Wux, df.Constant(0.0), boundary_left)
bc_ux_right = df.DirichletBC(Wux, df.Constant(0.0), boundary_right)

bc_uy_left  = df.DirichletBC(Wuy, df.Constant(0.0), boundary_left)
bc_uy_right = df.DirichletBC(Wuy, df.Constant(0.0), boundary_right)


bcs = [bc_phi_left, bc_phi_right, bc_ux_left, bc_ux_right, bc_uy_left, bc_uy_right] # no-flux on top, bottom boundary

###############
J = df.derivative(F, w, dw)
